In [1]:
import numpy as np
import data
from sklearn.decomposition import PCA
from sklearn.neighbors import LocalOutlierFactor as LOF
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from nn import NNEstimator
from sklearn.model_selection import train_test_split, GridSearchCV
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
train_data = data.Dataset('train.csv')
#train_data.to_one_hot()


In [3]:
train_x,test_x,train_y,test_y = train_test_split(train_data.x,train_data.y,test_size=0.1)


In [7]:
pipe = Pipeline([
    ('pca',PCA()),
    ('lda',LinearDiscriminantAnalysis()),
    ('nn',NNEstimator())
])

param_grid = {
    'pca__n_components': [256,512,784],
    'nn__weight_decay': [0.0,10e-3,10e-4,10e-5],
    'nn__hidden_size': [20,32,64,100],
    'nn__activation': ['relu','tanh'],
    'nn__optimizer': ['adam','sgd'],
}

grid = GridSearchCV(pipe,param_grid,cv=5,verbose=3,n_jobs=-1)
grid.fit(train_x,train_y)


Fitting 5 folds for each of 54 candidates, totalling 270 fits
[CV 1/5] END lda__n_components=10, lr__C=0.1, lr__solver=lbfgs, pca__n_components=256;, score=0.717 total time=   1.7s
[CV 3/5] END lda__n_components=10, lr__C=0.1, lr__solver=lbfgs, pca__n_components=256;, score=0.717 total time=   2.6s
[CV 2/5] END lda__n_components=10, lr__C=0.1, lr__solver=lbfgs, pca__n_components=256;, score=0.648 total time=   2.6s
[CV 4/5] END lda__n_components=10, lr__C=0.1, lr__solver=lbfgs, pca__n_components=256;, score=0.758 total time=   2.9s
[CV 5/5] END lda__n_components=10, lr__C=0.1, lr__solver=lbfgs, pca__n_components=256;, score=0.780 total time=   2.8s
[CV 2/5] END lda__n_components=10, lr__C=0.1, lr__solver=lbfgs, pca__n_components=512;, score=0.644 total time=   6.7s
[CV 3/5] END lda__n_components=10, lr__C=0.1, lr__solver=lbfgs, pca__n_components=512;, score=0.703 total time=   7.6s
[CV 1/5] END lda__n_components=10, lr__C=0.1, lr__solver=lbfgs, pca__n_components=512;, score=0.658 total

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('pca', PCA()),
                                       ('lda', LinearDiscriminantAnalysis()),
                                       ('lr',
                                        LogisticRegression(max_iter=10000))]),
             n_jobs=-1,
             param_grid={'lda__n_components': [10, 15, None],
                         'lr__C': [0.1, 1, 10], 'lr__solver': ['lbfgs', 'sag'],
                         'pca__n_components': [256, 512, 784]},
             verbose=3)

In [8]:
grid.best_params_

{'lda__n_components': None,
 'lr__C': 0.1,
 'lr__solver': 'sag',
 'pca__n_components': 256}

In [10]:
pipe = Pipeline([
    ('pca',PCA(n_components=256)),
    ('lda',LinearDiscriminantAnalysis()),
    ('lr',LogisticRegression(max_iter=10000,C=0.1,solver='sag'))
])

pipe.fit(train_data.x,train_data.y)

Pipeline(steps=[('pca', PCA(n_components=256)),
                ('lda', LinearDiscriminantAnalysis()),
                ('lr',
                 LogisticRegression(C=0.1, max_iter=10000, solver='sag'))])

In [9]:
grid.score(test_x,test_y)

0.8114754098360656

In [86]:
nn.train(train_data.x,train_data.y,validation_split=0.1,epochs=200,weight_decay=1.5*10e-6)

Epoch [1/200], Loss: 2.9765
Epoch [2/200], Loss: 2.9422
Epoch [3/200], Loss: 2.8863
Epoch [4/200], Loss: 2.8386
Epoch [5/200], Loss: 2.8149
Epoch [6/200], Loss: 2.7104
Epoch [7/200], Loss: 2.6909
Epoch [8/200], Loss: 2.5839
Epoch [9/200], Loss: 2.5155
Epoch [10/200], Loss: 2.4122
Epoch [11/200], Loss: 2.3403
Epoch [12/200], Loss: 2.4247
Epoch [13/200], Loss: 2.3476
Epoch [14/200], Loss: 2.4392
Epoch [15/200], Loss: 2.4833
Epoch [16/200], Loss: 2.3603
Epoch [17/200], Loss: 2.2872
Epoch [18/200], Loss: 2.2962
Epoch [19/200], Loss: 2.2487
Epoch [20/200], Loss: 2.2871
Epoch [21/200], Loss: 2.2569
Epoch [22/200], Loss: 2.2725
Epoch [23/200], Loss: 2.2568
Epoch [24/200], Loss: 2.3059
Epoch [25/200], Loss: 2.2744
Epoch [26/200], Loss: 2.1937
Epoch [27/200], Loss: 2.2382
Epoch [28/200], Loss: 2.1617
Epoch [29/200], Loss: 2.1619
Epoch [30/200], Loss: 2.1594
Epoch [31/200], Loss: 2.2384
Epoch [32/200], Loss: 2.1522
Epoch [33/200], Loss: 2.1970
Epoch [34/200], Loss: 2.1765
Epoch [35/200], Loss: 2

tensor(0.8430)

In [87]:
import torch
m = nn.Net(train_data.x.shape[1])
m.load_state_dict(torch.load('model.pth'))


<All keys matched successfully>

In [88]:
m(torch.from_numpy(test_x).float()).argmax(1).numpy()

RuntimeError: mat1 and mat2 shapes cannot be multiplied (121x256 and 300x20)

In [71]:
print(clf.best_score_)
print(clf.best_params_)

0.6513184843830006
{'bootstrap': False, 'criterion': 'entropy', 'max_depth': None, 'min_samples_split': 8}


In [11]:
test_data = data.Dataset('test.csv')
pred = pipe.predict(test_data.x)
data.write_to_csv('submission.csv',pred,train_data.get_cat_to_label())